In [1]:
import matplotlib
from matplotlib import pylab
import numpy as np
path2data = ".\\data\\"

In [2]:
def load_img_label(labels_df, id_):
    
    imgName = labels_df["imgName"]
    if imgName [id_][0] == "A":
        perfix = "AMD"
    else :
        perfix = "Non_AMD"
    fullPath2img = os.path.join(path2data,"Training400",
                                perfix,imgName[id_])
    img = Image.open(fullPath2img)
    x = labels_df["Fovea_X"][id_]
    y = labels_df["Fovea_Y"][id_]
    
    label = (x,y)
    return img, label

# Show images and labels
def show_img_label(img, label, w_h=(50,50),thickness=2):
    
    w,h = w_h
    cx,cy = label
    draw = ImageDraw.Draw(img)
    
    draw.rectangle([(cx-w/2, cy-h/2), (cx+w/2,cy+h/2)],outline="green",width=thickness)
    plt.imshow(np.asarray(img))
# Preprocess Function
def resize_img_label(img, label = (0.0,0.0), target_size = (256,256)):
    
    w_orig, h_orig = img.size
    w_target, h_target = target_size
    
    cx,cy = label
    image_new = TF.resize(img,target_size)
    label_new = cx/w_orig*w_target, cy/h_orig*h_target
    
    return image_new,label_new



def random_hflip(img, label):
    
    w,h = img.size
    x,y = label
    
    image = TF.hflip(img)
    label = w-x, y
    return image, label

def random_vflip(img, label):
    
    w,h = img.size
    x,y = label
    
    image = TF.vflip(img)
    label = x, w-y
    return image, label

def random_shift(img, label, max_translate=(0.2,0.2)):
    
    np.random.seed(1)
    w,h = img.size
    max_t_w, max_t_h = max_translate
    cx, cy = label
    trans_coef = np.random.rand()*2 - 1
    w_t = int (trans_coef*max_t_w*w)
    h_t = int(trans_coef*max_t_h*h)
    
    image = TF.affine(img, translate=(w_t,h_t),shear = 0,angle=0,scale = 1)
    label = cx+w_t , cy+h_t
    
    return image, label

def transformer(image,label, params):
    
    
    image, label = resize_img_label(image,label,params["target_size"])
    
    if random.random() < params["p_hflip"]:
        image, label = random_hflip(image, label)
    
    if random.random() < params["p_vflip"]:
        image, label = random_vflip(image, label)
    
    if random.random() < params["p_shift"]:
        image, label = random_shift(image, label, params["max_translate"])
    
    image = TF.to_tensor(image) 
    return image, label

def rescale_label(a,b):
    div = [ai*bi for ai,bi in zip(a,b)]
    return div


def show(img,label=None):
    npimg = img.numpy().transpose((1,2,0))
    plt.imshow(npimg)
    if label is not None:
        label=rescale_label(label,img.shape[1:])
        x,y=label
        plt.plot(x,y,'b+',markersize=20)

In [3]:
import os
import pandas as pd


path2labels = os.path.join(path2data, "Training400","Fovea_location.xlsx" )

labels_df = pd.read_excel(path2labels, index_col = "ID")
# In order to show head function output in "spyder" IDE should be use "print"
# In order to show head function output in "notebook" IDE can be use without "print"
print(labels_df.head())
print(labels_df.tail())

#Select a random set of image ids
nrows, ncolmn = 2,3
imgName = labels_df["imgName"]
ids = labels_df.index
rndIds = np.random.choice(ids, nrows*ncolmn)
print(rndIds)
#### The scatter plot of the Fovea_X and Fovea_Y
import seaborn as sns
AorN = [imn[0] for imn in labels_df.imgName]
sns.scatterplot(x=labels_df['Fovea_X'], y=labels_df.Fovea_Y, hue=AorN)
#### Show a few sample images
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

np.random.seed(2019)

plt.rcParams['figure.figsize'] = (15,9)
plt.subplots_adjust(wspace=0, hspace=0.3)
nrows,ncols = 2,3

imgName = labels_df["imgName"]
# genrate random
#ids = np.random.randint(0,len(imgName), nrows*ncols)
ids = labels_df.index
rndIds = np.random.choice(ids, nrows*ncols)
print(ids)
  
for i, id_ in enumerate(rndIds):
    
    img,label = load_img_label(labels_df,id_)
    print(img.size,label)
    plt.subplot(nrows,ncols,i+1)
    show_img_label(img, label, w_h=(150,150),thickness=20)
    plt.title(imgName[id_])  

FileNotFoundError: [Errno 2] No such file or directory: '.\\data\\Training400\\Fovea_location.xlsx'

In [ ]:
h_list, w_list = [],[]

for id_ in ids:
    
    if imgName[id_][0] == "A":
        prefix ="AMD"
    else:
        prefix ="Non_AMD"
    fullPath2img = os.path.join(path2data,"Training400",prefix,imgName[id_])
    img = Image.open(fullPath2img)
    h,w = img.size
    h_list.append(h)
    w_list.append(w)
sns.displot(a=h_list, kde=False)
sns.distplot(a=h_list, kde=False)
sns.distplot(a=w_list, kde=False)

In [ ]:
import random
import torchvision.transforms.functional as TF

np.random.seed(0)
random.seed(0)

image, label = load_img_label(labels_df,1)
params = {
        "target_size": (256,256),
        "p_hflip": 1.0,
        "p_vflip": 1.0,
        "p_shift": 1.0,
        "max_translate": (0.2,0.2),
        }

img_t, label_t = transformer(image, label,params)

plt.subplot(1,2,1)
show_img_label(image, label, w_h=(150,150),thickness=20)
plt.subplot(1,2,2)
show_img_label(TF.to_pil_image(img_t), label_t)

In [ ]:
from torch.utils.data import Dataset
from PIL import Image

class AMD_dataset (Dataset):
   
    
    def __init__(self, path2data, transform, trans_param):
        
        path2labels=os.path.join(path2data,"Training400","Fovea_location.xlsx")
        
        labels_df = pd.read_excel(path2labels, index_col="ID")
        self.labels = labels_df[["Fovea_X","Fovea_Y"]].values
        
        self.imgName = labels_df["imgName"]
        self.ids = labels_df.index
        
        self.fullPath2image = [0]*len(self.ids)
        
        for id_ in self.ids:
            
            if self.imgName[id_][0]=="A":
                prefix ="AMD"
            else:
                prefix ="Non_AMD"
            
            self.fullPath2image[id_-1]=os.path.join(path2data,"Training400",prefix,self.imgName[id_])
        self.transform = transform
        self.trans_params = trans_param
            
    def __len__(self): 
        return len(self.labels)
    
    def __getitem__(self, idx):
        
       image = Image.open(self.fullPath2image[idx])
       label = self.labels[idx]
       image, label = self.transform(image, label, self.trans_params)
       
       return image, label
   
    #AMD_dataset.__init__=__init__
    #AMD_dataset.__getitem__=__getitem__

trans_params_train = {
        
        "target_size": (256,256),
        "p_hflip": 0.5,
        "p_vflip": 0.5,
        "p_shift": 0.5,
        "max_translate": (0.2,0.2),
        "p_brightness": 0.5,
        "brightness_factor": 0.2,
        "p_contrast": 0.2,
        "p_gamma": 0.5,
        "gamma": 0.2,
        "scale_label": True,
        }
trans_params_val = {
        
        "target_size": (256,256),
        "p_hflip": 0,
        "p_vflip": 0,
        "p_shift": 0,
        "p_brightness": 0,
        "brightness_factor": 0,
        "p_contrast": 0,
        "p_gamma": 0,
        "gamma": 0,
        "scale_label": True,  
        }

amd_ds1=AMD_dataset(path2data,transformer,trans_params_train)
amd_ds2=AMD_dataset(path2data,transformer,trans_params_val)

In [ ]:
from sklearn.model_selection import ShuffleSplit

sss = ShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
indices = range(len(amd_ds1))

for train_index, val_index in sss.split(indices):
    
    print(len(train_index))
    print('-'*10)
    print(len(val_index))

#Define the training and validation datasets
from torch.utils.data import Subset

train_ds=Subset(amd_ds1,train_index)
print(len(train_ds))

val_ds=Subset(amd_ds2,val_index)
print(len(val_ds))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline


np.random.seed(0)

plt.figure(figsize=(5,5))
for img,label in train_ds:
    show(img,label)
    break